#### **Creation of Simple Pos Tagging** 

In [ ]:
import pandas as pd
import conllu
import math

In [ ]:
#loading the training corpus

# Open the .conllu file
with open("train.conllu", "r", encoding="utf-8") as file:
    # Read the contents of the file
    data = file.read()

In [ ]:
#loading the test corpus
#open the .conllu file

with open("test.conllu","r",encoding="utf-8") as f:
    #read the content
    content = f.read()
with open("test.pos","w",encoding="utf-8") as file:
    sen = conllu.parse(content)
    for s in sen:
        for t in s:
            word=t["form"]
            ps = t["upostag"]
            file.write(word + '\t' + ps +'\n')

In [ ]:
# Parse the .conllu data
import conllu
sentences = conllu.parse(data)

In [ ]:
with open("train.pos", 'w', encoding='utf-8') as file:
    for sentence in sentences:
    # Iterate over each token in the sentence
        for token in sentence:
        # Access token properties such as word form, lemma, part of speech, etc.
            word_form = token["form"]
            pos = token["upostag"]
            file.write(word_form + '\t' + pos +'\n')

In [ ]:
with open(r"C:\Users\pc\Downloads\ANTILLES-main\ANTILLES-main\ANTILLES\train.conllu", "r", encoding="utf-8") as f:
    train_data = f.read()
with open(r"C:\Users\pc\Downloads\ANTILLES-main\ANTILLES-main\ANTILLES\dev.conllu", "r", encoding="utf-8") as f:
    test_data = f.read()
train = {}
sentences = conllu.parse(train_data)

for sentence in sentences:
    for token in sentence:
        train[token['form']] = token['upos']
test = {}
sentences = conllu.parse(test_data)

for sentence in sentences:
    for token in sentence:
        test[token['form']] = token['upos']
vocab_l=list()
sentences = conllu.parse(train_data)

for sentence in sentences:
    for token in sentence:
        vocab_l.append(token['form'])
vocab_l=vocab_l+list(test.keys())

In [ ]:
len(vocab_l)

373634

START

In [ ]:
#loding train data
with open("train.pos", 'r',encoding='utf-8') as f:
    training_corpus = f.readlines()

print(f"A few items of the training corpus list")
print(training_corpus[0:5])

A few items of the training corpus list
['Les\tDET\n', 'commotions\tNFP\n', 'cérébrales\tADJFP\n', 'sont\tAUX\n', 'devenu\tVPPMS\n']


In [ ]:
# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(set(vocab_l))): 
    vocab[word] = i       
    
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 20:
        break

Vocabulary dictionary, key is the word, value is a unique integer
!:0
":1
#:2
$:3
%:4
&:5
':6
'':7
'06:8
'07:9
'900:10
'upa'upa:11
(:12
):13
*:14
+:15
+0,5:16
+0,6:17
+1:18
+1,8:19
+14:20


In [ ]:
len(vocab)

44929

In [ ]:
# load in the test corpus
with open("test.pos", 'r') as f:
    y = f.readlines()

print("A sample of the test corpus")
print(y[0:10])

A sample of the test corpus
['Je\tPPER1S\n', 'sens\tVERB\n', "qu'\tCOSUB\n", 'entre\tPREP\n', 'Ã§a\tPDEMMS\n', 'et\tCOCO\n', 'les\tDET\n', 'films\tNMP\n', 'de\tPREP\n', 'mÃ©decins\tNMP\n']


In [ ]:
# testing words
with open("test.pos","r") as f:
    lines=f.readlines()

    first_column_words = []

# Parcourir chaque ligne et extraire le mot de la première colonne
for line in lines:
    # Diviser la ligne en colonnes en utilisant '\t' comme séparateur
    columns = line.split('\t')

    # Vérifier si le nombre de colonnes est suffisant
    if len(columns) >= 1:
        # Ajouter le mot de la première colonne à la liste
        first_column_words.append(columns[0])

# Ouvrir le fichier de sortie en écriture
with open('testing.words', 'w') as output_file:
    # Écrire les mots de la première colonne dans le fichier de sortie
    for w in first_column_words:
        output_file.write(w + '\n')
        


In [ ]:

def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep
def assign_unk(word):
    return "<UNK>"

def get_word_tag(line, vocabulary):
    if not line.strip():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        parts = line.split("\t", 1)
        if len(parts) != 2:
            return None, None
        word, tag = parts
        if word not in vocabulary: 
            word = assign_unk(word)
        return word, tag.strip()

In [ ]:
#corpus without tags, preprocessed
_, prep = preprocess(vocab, "testing.words")     

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of the test_corpus: ')
len(prep)

The length of the preprocessed test corpus:  10298
This is a sample of the test_corpus: 


10298

**TRAINING**

In [ ]:
def create_dictionaries(training_corpus, vocab):
    """
    Input: 
        training_corpus: a corpus where each line has a word followed by its tag.
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output: 
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        transition_counts: a dictionary where the keys are (prev_tag, tag) and the values are the counts
        tag_counts: a dictionary where the keys are the tags and the values are the counts
    """

    from collections import defaultdict
    # initialize the dictionaries using defaultdict
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '--s--' 
    
    # use 'i' to track the line number in the corpus
    i = 0 
    
    # Each item in the training corpus contains a word and its POS tag
    # Go through each word and its tag in the training corpus
    for word_tag in training_corpus: 
        
        # Increment the word_tag count
        i += 1
        
        # Every 50,000 words, print the word count
        if i % 50000 == 0:
            print(f"word count = {i}")
            
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # get the word and tag using the get_word_tag helper function (imported from utils_pos.py)
        word, tag = get_word_tag(word_tag,vocab) 
        
        # Increment the transition count for the previous word and tag
        transition_counts[(prev_tag, tag)] += 1
        
        # Increment the emission count for the tag and word
        emission_counts[(tag, word)] += 1

        # Increment the tag count
        tag_counts[tag] += 1

        # Set the previous tag to this tag (for the next iteration of the loop)
        prev_tag = tag
        
        ### END CODE HERE ###
        
    return emission_counts, transition_counts, tag_counts

In [ ]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000


In [ ]:
emission_counts

defaultdict(int,
            {('DET', 'Les'): 893,
             ('NFP', 'commotions'): 1,
             ('ADJFP', 'cérébrales'): 1,
             ('AUX', 'sont'): 1024,
             ('VPPMS', 'devenu'): 38,
             ('ADV', 'si'): 24,
             ('ADJFP', 'courantes'): 4,
             ('PREP', 'dans'): 2612,
             ('PDEMMS', 'ce'): 492,
             ('NMS', 'sport'): 16,
             ('COSUB', "qu'"): 588,
             ('PINDMS', 'on'): 349,
             ('PPOBJMS', 'les'): 56,
             ('VERB', 'considére'): 1,
             ('ADV', 'presque'): 32,
             ('PREP', 'comme'): 462,
             ('DETFS', 'la'): 8581,
             ('NFS', 'routine'): 1,
             ('YPFOR', '.'): 13360,
             ('DET', "L'"): 740,
             ('NFS', 'œuvre'): 69,
             ('AUX', 'est'): 4509,
             ('VPPFS', 'située'): 175,
             ('NFS', 'galerie'): 12,
             ('_', 'des'): 2908,
             ('PREP', 'de'): 23425,
             ('DET', 'les'): 7696,
  

In [ ]:
tag_counts

defaultdict(int,
            {'DET': 22331,
             'NFP': 6543,
             'ADJFP': 2671,
             'AUX': 11407,
             'VPPMS': 5560,
             'ADV': 13158,
             'PREP': 56504,
             'PDEMMS': 1711,
             'NMS': 26266,
             'COSUB': 2623,
             'PINDMS': 853,
             'PPOBJMS': 1249,
             'VERB': 18705,
             'DETFS': 9731,
             'NFS': 22597,
             'YPFOR': 13360,
             'VPPFS': 2185,
             '_': 9687,
             'PUNCT': 25656,
             'DETMS': 13656,
             'PROPN': 19728,
             'ADJMS': 7318,
             'PPER3FS': 880,
             'ADJFS': 7084,
             'COCO': 8973,
             'NMP': 9595,
             'PREL': 2644,
             'PPER1S': 488,
             'ADJMP': 2900,
             'VPPMP': 1229,
             'DINTMS': 3745,
             'PPER3MS': 3499,
             'PPER3MP': 464,
             'PREF': 2053,
             'ADJ': 952,
          

In [ ]:
transition_counts

defaultdict(int,
            {('--s--', 'DET'): 1,
             ('DET', 'NFP'): 4162,
             ('NFP', 'ADJFP'): 1452,
             ('ADJFP', 'AUX'): 89,
             ('AUX', 'VPPMS'): 2918,
             ('VPPMS', 'ADV'): 224,
             ('ADV', 'ADJFP'): 198,
             ('ADJFP', 'PREP'): 437,
             ('PREP', 'PDEMMS'): 430,
             ('PDEMMS', 'NMS'): 712,
             ('NMS', 'COSUB'): 172,
             ('COSUB', 'PINDMS'): 52,
             ('PINDMS', 'PPOBJMS'): 71,
             ('PPOBJMS', 'VERB'): 858,
             ('VERB', 'ADV'): 2522,
             ('ADV', 'PREP'): 2425,
             ('PREP', 'DETFS'): 5679,
             ('DETFS', 'NFS'): 7774,
             ('NFS', 'YPFOR'): 1556,
             ('YPFOR', 'DET'): 2005,
             ('DET', 'NFS'): 4441,
             ('NFS', 'AUX'): 830,
             ('AUX', 'VPPFS'): 899,
             ('VPPFS', 'PREP'): 1397,
             ('NFS', '_'): 2291,
             ('_', 'PREP'): 9687,
             ('PREP', 'DET'): 11875,


In [ ]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 66
View these POS tags (states)
['ADJ', 'ADJFP', 'ADJFS', 'ADJMP', 'ADJMS', 'ADV', 'AUX', 'CHIF', 'COCO', 'COSUB', 'DET', 'DETFS', 'DETMS', 'DINTFS', 'DINTMS', 'INTJ', 'MOTINC', 'NFP', 'NFS', 'NMP', 'NMS', 'NOUN', 'NUM', 'PART', 'PDEMFP', 'PDEMFS', 'PDEMMP', 'PDEMMS', 'PINDFP', 'PINDFS', 'PINDMP', 'PINDMS', 'PINTFS', 'PPER1S', 'PPER2S', 'PPER3FP', 'PPER3FS', 'PPER3MP', 'PPER3MS', 'PPOBJFP', 'PPOBJFS', 'PPOBJMP', 'PPOBJMS', 'PREF', 'PREFP', 'PREFS', 'PREL', 'PRELFP', 'PRELFS', 'PRELMP', 'PRELMS', 'PREP', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'VPPFP', 'VPPFS', 'VPPMP', 'VPPMS', 'VPPRE', 'X', 'XFAMIL', 'YPFOR', '_']


In [ ]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'comparable':
        print (tup, cnt) 

transition examples: 
(('--s--', 'DET'), 1)
(('DET', 'NFP'), 4162)
(('NFP', 'ADJFP'), 1452)

emission examples: 
(('ADJMP', 'autres'), 135)
(('ADJMP', 'petits'), 29)
(('NMP', 'fruits'), 9)

ambiguous word example: 
('ADJMS', 'comparable') 1
('ADJFS', 'comparable') 4


**TESTING**

In [ ]:
def predict_pos(prep, y, emission_counts, vocab, states):
    '''
    Input: 
        prep: a preprocessed version of 'y'. A list with the 'word' component of the tuples.
        y: a corpus composed of a list of tuples where each tuple consists of (word, POS)
        emission_counts: a dictionary where the keys are (tag,word) tuples and the value is the count
        vocab: a dictionary where keys are words in vocabulary and value is an index
        states: a sorted list of all possible tags for this assignment
    Output: 
        accuracy: Number of times you classified a word correctly
    '''
    
    # Initialize the number of correct predictions to zero
    num_correct = 0
    
    # Get the (tag, word) tuples, stored as a set
    all_words = set(emission_counts.keys())
    
    # Get the number of (word, POS) tuples in the corpus 'y'
    total = len(y)
    for word, y_tup in zip(prep, y): 

        # Split the (word, POS) string into a list of two items
        y_tup_l = y_tup.split()
        
        # Verify that y_tup contain both word and POS
        if len(y_tup_l) == 2:
            
            # Set the true POS label for this word
            true_label = y_tup_l[1]

        else:
            # If the y_tup didn't contain word and POS, go to next word
            continue
    
        count_final = 0
        pos_final = ''
        
        # If the word is in the vocabulary...
        if word in vocab:
            for pos in states:

            ### START CODE HERE (Replace instances of 'None' with your code) ###
                        
                # define the key as the tuple containing the POS and word
                key = (pos,word)

                # check if the (pos, word) key exists in the emission_counts dictionary
                if key in emission_counts: # complete this line

                # get the emission count of the (pos,word) tuple 
                    count = emission_counts[key]

                    # keep track of the POS with the largest count
                    if count>count_final: # complete this line

                        # update the final count (largest count)
                        count_final = count

                        # update the final POS
                        pos_final = pos

            # If the final POS (with the largest count) matches the true POS:
            if pos_final == true_label: # complete this line
                
                # Update the number of correct predictions
                num_correct += 1
            
    ### END CODE HERE ###
    accuracy = num_correct / total
    
    return accuracy

In [ ]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.7606


HIDDEN MARKOV  MODELS FOR POS

In [ ]:
#generating matrices
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Input: 
        alpha: number used for smoothing
        tag_counts: a dictionary mapping each tag to its respective count
        transition_counts: transition count for the previous word and tag
    Output:
        A: matrix of dimension (num_tags,num_tags)
    '''
    # Get a sorted list of unique POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Count the number of unique POS tags
    num_tags = len(all_tags)
    
    # Initialize the transition matrix 'A'
    A = np.zeros((num_tags,num_tags))
    
    # Get the unique transition tuples (previous POS, current POS)
    trans_keys = set(transition_counts.keys())
    

    # Go through each row of the transition matrix A
    for i in range(num_tags):
        
        # Go through each column of the transition matrix A
        for j in range(num_tags):

            # Initialize the count of the (prev POS, current POS) to zero
            count = 0
        
            # Define the tuple (prev POS, current POS)
            # Get the tag at position i and tag at position j (from the all_tags list)
            key = (all_tags[i],all_tags[j])

            # Check if the (prev POS, current POS) tuple 
            # exists in the transition counts dictionaory
            if transition_counts: #complete this line
                
                # Get count from the transition_counts dictionary 
                # for the (prev POS, current POS) tuple
                count = transition_counts[key]
                
            # Get the count of the previous tag (index position i) from tag_counts
            count_prev_tag = tag_counts[all_tags[i]]
            
            # Apply smoothing using count of the tuple, alpha, 
            # count of previous tag, alpha, and number of total tags
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)

    
    return A

In [ ]:
import numpy as np
import pandas as pd
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[2,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

A at row 0, col 0: 0.013655566
A at row 3, col 1: 0.0000
View a subset of transition matrix A
          PINDMP    PINDMS    PINTFS    PPER1S    PPER2S
PINDMP  0.000012  0.000012  0.000012  0.000012  0.000012
PINDMS  0.000001  0.000001  0.000001  0.000001  0.000001
PINTFS  0.000484  0.000484  0.000484  0.000484  0.000484
PPER1S  0.000002  0.000002  0.000002  0.000002  0.000002
PPER2S  0.000066  0.000066  0.000066  0.000066  0.000066


In [ ]:
len(prep)

10298

In [ ]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Input: 
        alpha: tuning parameter used in smoothing 
        tag_counts: a dictionary mapping each tag to its respective count
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        B: a matrix of dimension (num_tags, len(vocab))
    '''
    
    # get the number of POS tag
    num_tags = len(tag_counts)
    
    # Get a list of all POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Get the total number of unique words in the vocabulary
    num_words = len(vocab)
    
    # Initialize the emission matrix B with places for
    # tags in the rows and words in the columns
    B = np.zeros((num_tags, num_words))
    
    # Get a set of all (POS, word) tuples 
    # from the keys of the emission_counts dictionary
    emis_keys = set(list(emission_counts.keys()))
    

    # Go through each row (POS tags)
    for i in range(num_tags): # complete this line
        
        # Go through each column (words)
        for j in range(num_words): # complete this line

            # Initialize the emission count for the (POS tag, word) to zero
            count = 0
                    
            # Define the (POS tag, word) tuple for this row and column
            key =  (all_tags[i],vocab[j])

            # check if the (POS tag, word) tuple exists as a key in emission counts
            if key in emission_counts.keys(): # complete this line
        
                # Get the count of (POS tag, word) from the emission_counts d
                count = emission_counts[key]
                
            # Get the count of the POS tag
            count_tag = tag_counts[all_tags[i]]
                
            # Apply smoothing and store the smoothed value 
            # into the emission matrix B for this row and column
            B[i,j] = (count + alpha) / (count_tag+ alpha*num_words)

    return B

In [ ]:
# creating your emission probability matrix. this takes a few minutes to run. 
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")



View Matrix position at row 0, column 0: 0.000001003
View Matrix position at row 3, column 1: 0.000000340


In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: initialize
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Input: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
    # Get the total number of unique POS tags
    num_tags = len(tag_counts)
    
    # Initialize best_probs matrix 
    # POS tags in the rows, number of words in the corpus as the columns
    best_probs = np.zeros((num_tags, len(corpus)))
    
    # Initialize best_paths matrix
    # POS tags in the rows, number of words in the corpus as columns
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    # Define the start token
    s_idx = states.index("_")
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    
    # Go through each of the POS tags
    for i in range(num_tags): # complete this line
        
        # Handle the special case when the transition from start token to POS tag i is zero
        if A[s_idx,i] == 0: # complete this line
            
            # Initialize best_probs at POS tag 'i', column 0, to negative infinity
            best_probs[i,0] = float('-inf')
        
        # For all other cases when transition from start token to POS tag i is non-zero:
        else:
            
            # Initialize best_probs at POS tag 'i', column 0
            # Check the formula in the instructions above
            best_probs[i,0] = math.log(A[s_idx,i]) + math.log(B[i, vocab[corpus[0]]])
                        
    ### END CODE HERE ### 
    return best_probs, best_paths

In [ ]:

best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [ ]:
# Test the function
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,4]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -29.8987
best_paths[2,4]: 0.0000


In [ ]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_forward
def viterbi_forward(A, B, test, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transition and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    # Get the number of unique POS tags (which is the num of rows in best_probs)
    num_tags = best_probs.shape[0]
    
    # Go through every word in the corpus starting from word 1
    # Recall that word 0 was initialized in `initialize()`
    for i in range(1, len(test)): 
        
        # Print number of words processed, every 5000 words
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
            
        ### START CODE HERE (Replace instances of 'None' with your code EXCEPT the first 'best_path_i = None') ###
        # For each unique POS tag that the current word can be
        for j in range(num_tags): # complete this line
            
            # Initialize best_prob for word i to negative infinity
            best_prob_i = float("-inf")
            
            # Initialize best_path for current word i to None
            best_path_i = None

            # For each POS tag that the previous word can be:
            for k in range(num_tags): 
                # complete this line
            
                # Calculate the probability = 
                # best probs of POS tag k, previous word i-1 + 
                # log(prob of transition from POS k to POS j) + 
                # log(prob that emission of POS j is word i)
                word = test[i]
                if word in vocab:
                        word_index = vocab[word]
                else:
    # Assign a specific index for unknown words or treat them as a separate category
    # For example, you can assign them the last index in the vocabulary
                        word_index = len(vocab) - 1

# Calculate the probability using the updated word_index
                prob = best_probs[k, i - 1] + math.log(A[k, j]) + math.log(B[j, word_index])

                # check if this path's probability is greater than
                # the best probability up to and before this point
                if prob > best_prob_i: # complete this line
                    
                    # Keep track of the best probability
                    best_prob_i = prob
                    
                    # keep track of the POS tag of the previous word
                    # that is part of the best path.  
                    # Save the index (integer) associated with 
                    # that previous word's POS tag
                    best_path_i = k

            # Save the best probability for the 
            # given current word's POS tag
            # and the position of the current word inside the corpus
            best_probs[j,i] = best_prob_i
            
            # Save the unique integer ID of the previous POS tag
            # into best_paths matrix, for the POS tag of the current word
            # and the position of the current word inside the corpus.
            best_paths[j,i] = best_path_i

        ### END CODE HERE ###
    return best_probs, best_paths

In [ ]:
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000


In [ ]:
# Test this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}")

best_probs[0,1]: -39.4696
best_probs[0,4]: -62.8439


In [ ]:
# UNQ_C7 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: viterbi_backward
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    # Get the number of words in the corpus
    # which is also the number of columns in best_probs, best_paths
    m = best_paths.shape[1] 
    
    # Initialize array z, same length as the corpus
    z = [None] * m
    
    # Get the number of unique POS tags
    num_tags = best_probs.shape[0]
    
    # Initialize the best probability for the last word
    best_prob_for_last_word = float('-inf')
    
    # Initialize pred array, same length as corpus
    pred = [None] * m
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    ## Step 1 ##
    
    # Go through each POS tag for the last word (last column of best_probs)
    # in order to find the row (POS tag integer ID) 
    # with highest probability for the last word
    for k in range(num_tags): # complete this line

        # If the probability of POS tag at row k 
        # is better than the previosly best probability for the last word:
        if best_probs[k, m - 1] > best_prob_for_last_word: # complete this line
            
            # Store the new best probability for the last word
            best_prob_for_last_word = best_probs[k, m - 1]
    
            # Store the unique integer ID of the POS tag
            # which is also the row number in best_probs
            z[m - 1] = k
            
    # Convert the last word's predicted POS tag
    # from its unique integer ID into the string representation
    # using the 'states' list
    # store this in the 'pred' array for the last word
    pred[m - 1] = states[k]
    
    ## Step 2 ##
    # Find the best POS tags by walking backward through the best_paths
    # From the last word in the corpus to the 0th word in the corpus
    for i in range(len(corpus) - 1, -1, -1): # complete this line
        
        # Retrieve the unique integer ID of
        # the POS tag for the word at position 'i' in the corpus
        pos_tag_for_word_i = z[i]
        
        # In best_paths, go to the row representing the POS tag of word i
        # and the column representing the word's position in the corpus
        # to retrieve the predicted POS for the word at position i-1 in the corpus
        z[i - 1] = best_paths[pos_tag_for_word_i, i]
        
        # Get the previous word's POS tag in string form
        # Use the 'states' list, 
        # where the key is the unique integer ID of the POS tag,
        # and the value is the string representation of that POS tag
        pred[i - 1] = states[z[i - 1]]
        
     ### END CODE HERE ###
    return pred

In [ ]:
# Run and test your function
pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['Nancy', ',', 'est', 'une', '<UNK>', '<UNK>'] 
 ['PROPN', 'PUNCT', 'AUX', 'DINTFS', 'NFS', 'PPOBJMS'] 

The prediction for pred[0:8] is: 
 ['PPER1S', 'VERB', 'COSUB', 'PREP', 'PPOBJMS', 'COCO', 'DET'] 
 ['Je', 'sens', "qu'", 'entre', '<UNK>', 'et', 'les']


In [ ]:
print('The third word is:', prep[8])
print('Your prediction is:', pred[8])
print('Your corresponding label y is: ', y[8])

The third word is: de
Your prediction is: PREP
Your corresponding label y is:  de	PREP



In [ ]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: compute_accuracy
def compute_accuracy(pred, y):
    '''
    Input: 
        pred: a list of the predicted parts-of-speech 
        y: a list of lines where each word is separated by a '\t' (i.e. word \t tag)
    Output: 
        
    '''
    num_correct = 0
    total = 0
    
    # Zip together the prediction and the labels
    for prediction, y in zip(pred, y):
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # Split the label into the word and the POS tag
        word_tag_tuple = y.split()
        
        # Check that there is actually a word and a tag
        # no more and no less than 2 items
        if len(word_tag_tuple)!=2: # complete this line
            continue

        # store the word and tag separately
        word, tag = word_tag_tuple

        # Check if the POS tag label matches the prediction
        if prediction == tag: # complete this line
            # count the number of times that the prediction
            # and label match
            num_correct += 1
            
        # keep track of the total number of examples (that have valid labels)
        total += 1
        
        ### END CODE HERE ###
    return num_correct / total

In [ ]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.7979
